In [ ]:
!pip install tpot

     |████████████████████████████████| 87 kB 2.6 MB/s 
     |████████████████████████████████| 166.7 MB 17 kB/s 
     |████████████████████████████████| 160 kB 64.7 MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=ddf09e672144cc3b4303ca5ac2a8a8bc98d0f38e241f7a3c63ce36a678d0d001
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
import sklearn.ensemble
import sklearn.metrics

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from tpot.builtins import ZeroCount, OneHotEncoder, StackingEstimator
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, make_union

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB


import time



/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Dados balanceados - Sem frequência

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving todos_dados_prf_balanceados_teste.csv to todos_dados_prf_balanceados_teste.csv
Saving todos_dados_prf_balanceados_treino.csv to todos_dados_prf_balanceados_treino.csv


In [ ]:
#Dados de treino
treino = pd.read_csv("todos_dados_prf_balanceados_treino.csv")
treino = treino.sample(frac=1)

treino.drop(treino.columns[[0,1,9]], axis=1, inplace=True)

#Dados de teste
teste = pd.read_csv("todos_dados_prf_balanceados_teste.csv")
teste = teste.sample(frac=1)

teste.drop(teste.columns[[0,1,9]], axis=1, inplace=True)

X = pd.concat([treino, teste])


Y = X[X.columns[7]]
X.drop(X.columns[[7]], axis=1, inplace=True)


X.head()

,dia_semana_num,turno_simples,tipo_pista_simples,categoria_sentido_via,tracado_via_simples,condicao_metereologica_simples,tipo_acidente_simples
422557,1,0,0.0,2,1,1,1
33078,0,1,1.0,1,1,1,1
307378,4,0,0.0,1,2,1,2
209264,7,1,0.0,1,2,1,1
426965,3,0,0.0,1,1,1,1


## TPOT Default

In [ ]:
## <medir tempo a partir daqui>
start_teste1 = time.time()

In [ ]:
imputer = SimpleImputer(strategy="median")
imputer.fit(X)
X = imputer.transform(X)
# testing_features = imputer.transform(testing_features)

# Average CV score on the training set was: 0.7208972774743891
exported_pipeline = make_pipeline(
    ZeroCount(),
    ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.8500000000000001, min_samples_leaf=7, min_samples_split=20, n_estimators=100)
)

#exported_pipeline.fit(training_features, training_target)
#results = exported_pipeline.predict(testing_features)

scoring = ['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(exported_pipeline, X, Y, scoring=scoring, cv=20, return_train_score=False)

sorted(scores.keys())

['fit_time',
 'score_time',
 'test_accuracy',
 'test_f1_macro',
 'test_precision_macro',
 'test_recall_macro']

In [ ]:
## <fim da medição de tempo>
end_teste1 = time.time()
print(end_teste1 - start_teste1)

In [ ]:
scores['test_accuracy']

array([0.65515482, 0.64715074, 0.65480404, 0.64960617, 0.64743774,
       0.64794796, 0.6515195 , 0.64651296, 0.64931917, 0.64737396,
       0.65043528, 0.6495424 , 0.65059473, 0.64820307, 0.64842629,
       0.65005262, 0.90634268, 0.91016646, 0.90184323, 0.90238536])

In [ ]:
scores['test_precision_macro']

array([0.65787943, 0.64970729, 0.65791521, 0.65252392, 0.65026337,
       0.65053163, 0.65408957, 0.64923836, 0.65187873, 0.65016627,
       0.65328751, 0.65239882, 0.65344736, 0.65118581, 0.65112509,
       0.65276872, 0.92118421, 0.92393476, 0.91802055, 0.91842119])

In [ ]:
scores['test_recall_macro']

array([0.65506784, 0.64706412, 0.65471119, 0.6495146 , 0.64734691,
       0.64786114, 0.65143393, 0.64642343, 0.64923315, 0.64728363,
       0.65034496, 0.64945174, 0.65050445, 0.64811008, 0.64833776,
       0.6499643 , 0.90622015, 0.91004598, 0.90171167, 0.90225444])

In [ ]:
scores['test_f1_macro']

array([0.65358201, 0.64555517, 0.65301138, 0.64783648, 0.64568595,
       0.64634838, 0.64998108, 0.64480586, 0.64775454, 0.64564097,
       0.64871734, 0.64780741, 0.64887914, 0.64637123, 0.64676588,
       0.6484076 , 0.90549151, 0.90941346, 0.90086308, 0.90142007])

In [ ]:
print("mean of test_accuracy : ", np.mean(scores['test_accuracy']))
print("mean of test_precision_macro : ", np.mean(scores['test_precision_macro']))
print("mean of test_recall_macro : ", np.mean(scores['test_recall_macro']))
print("mean of test_f1_macro : ", np.mean(scores['test_f1_macro']))

mean of test_accuracy :  0.7007409585685939
mean of test_precision_macro :  0.7059983890490342
mean of test_recall_macro :  0.7006442732694869
mean of test_f1_macro :  0.6992169272487555


## TPOT Sparse

In [ ]:
## <medir tempo a partir daqui>
start_teste1 = time.time()

In [ ]:
imputer = SimpleImputer(strategy="median")
imputer.fit(X)
X = imputer.transform(X)
#testing_features = imputer.transform(testing_features)

# Average CV score on the training set was: 0.7209271734364412
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.4, min_samples_leaf=3, min_samples_split=12, n_estimators=100)),
    OneHotEncoder(minimum_fraction=0.15),
    LogisticRegression(C=0.001, dual=False, penalty="l2")
)

#exported_pipeline.fit(training_features, training_target)
#results = exported_pipeline.predict(testing_features)

scoring = ['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(exported_pipeline, X, Y, scoring=scoring, cv=20, return_train_score=False)

sorted(scores.keys())

['fit_time',
 'score_time',
 'test_accuracy',
 'test_f1_macro',
 'test_precision_macro',
 'test_recall_macro']

In [ ]:
## <fim da medição de tempo>
end_teste1 = time.time()
print(end_teste1 - start_teste1)

In [ ]:
scores['test_accuracy']

array([0.65426193, 0.64711885, 0.65470838, 0.65030773, 0.64715074,
       0.64826685, 0.65190217, 0.64699129, 0.64947862, 0.64756529,
       0.65072228, 0.64966995, 0.64998884, 0.64861762, 0.64903218,
       0.65005262, 0.90580057, 0.91188851, 0.9041074 , 0.90627591])

In [ ]:
scores['test_precision_macro']

array([0.65686959, 0.64969774, 0.6577402 , 0.65331737, 0.64986212,
       0.65088201, 0.65453625, 0.64978143, 0.65206927, 0.65040419,
       0.65362665, 0.65253433, 0.6527516 , 0.65162842, 0.65178204,
       0.65282729, 0.92080024, 0.92517467, 0.91960764, 0.92115993])

In [ ]:
scores['test_recall_macro']

array([0.65417654, 0.64703186, 0.65461665, 0.65021499, 0.64706162,
       0.64817961, 0.65181568, 0.64690088, 0.64939213, 0.6474743 ,
       0.65063125, 0.64957921, 0.64989977, 0.64852434, 0.64894302,
       0.64996338, 0.90567733, 0.91177034, 0.90397888, 0.90615021])

In [ ]:
scores['test_f1_macro']

array([0.65274151, 0.64550939, 0.65295814, 0.64849657, 0.64546243,
       0.64665346, 0.65033259, 0.64525324, 0.64789811, 0.64580775,
       0.6489789 , 0.64793243, 0.64831566, 0.64677652, 0.64735131,
       0.64837352, 0.90493458, 0.91117781, 0.90319339, 0.9054212 ])

In [ ]:
print("mean of test_accuracy : ", np.mean(scores['test_accuracy']))
print("mean of test_precision_macro : ", np.mean(scores['test_precision_macro']))
print("mean of test_recall_macro : ", np.mean(scores['test_recall_macro']))
print("mean of test_f1_macro : ", np.mean(scores['test_f1_macro']))

mean of test_accuracy :  0.7011953861281108
mean of test_precision_macro :  0.7063526495583414
mean of test_recall_macro :  0.7010990998890596
mean of test_f1_macro :  0.6996784246835186


# Dados balanceados - Com frequência


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving todos_dados_prf_balanceados_com_frequencia_teste.csv to todos_dados_prf_balanceados_com_frequencia_teste.csv
Saving todos_dados_prf_balanceados_com_frequencia_treino.csv to todos_dados_prf_balanceados_com_frequencia_treino.csv


In [ ]:
#Dados de treino
treino = pd.read_csv("todos_dados_prf_balanceados_com_frequencia_treino.csv")
treino = treino.sample(frac=1)

treino.drop(treino.columns[[0,1,9]], axis=1, inplace=True)

#Dados de teste
teste = pd.read_csv("todos_dados_prf_balanceados_com_frequencia_teste.csv")
teste = teste.sample(frac=1)

teste.drop(teste.columns[[0,1,9]], axis=1, inplace=True)

X = pd.concat([treino, teste])


Y = X[X.columns[7]]
X.drop(X.columns[[7]], axis=1, inplace=True)


X.head()

,dia_semana_num,turno_simples,tipo_pista_simples,categoria_sentido_via,tracado_via_simples,condicao_metereologica_simples,tipo_acidente_simples,frequencia
35910,5,1,0.0,2,1,1,2,0.000005
208466,2,0,1.0,2,2,1,2,0.000172
180702,4,1,0.0,2,1,1,1,0.000458
428574,3,1,1.0,2,1,1,1,0.000241
384615,5,0,1.0,1,1,1,1,0.000158


## TPOT Default

In [ ]:
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy="median")
imputer.fit(X)
X = imputer.transform(X)

# Average CV score on the training set was: 0.7112249372184797
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.001, max_depth=8, min_child_weight=14, n_estimators=100, n_jobs=1, subsample=0.8500000000000001, verbosity=0)),
    DecisionTreeClassifier(criterion="gini", max_depth=5, min_samples_leaf=6, min_samples_split=2)
)

# exported_pipeline.fit(training_features, training_target)
# results = exported_pipeline.predict(testing_features)

scoring = ['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(exported_pipeline, X, Y, scoring=scoring, cv=20, return_train_score=False)

sorted(scores.keys())

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encode

['fit_time',
 'score_time',
 'test_accuracy',
 'test_f1_macro',
 'test_precision_macro',
 'test_recall_macro']

In [ ]:
print("mean of test_accuracy : ", np.mean(scores['test_accuracy']))
print("mean of test_precision_macro : ", np.mean(scores['test_precision_macro']))
print("mean of test_recall_macro : ", np.mean(scores['test_recall_macro']))
print("mean of test_f1_macro : ", np.mean(scores['test_f1_macro']))

mean of test_accuracy :  0.7113047865713821
mean of test_precision_macro :  0.7175263490912949
mean of test_recall_macro :  0.7111946365218481
mean of test_f1_macro :  0.7091629149922758


## TPOT Sparse

In [ ]:
imputer = SimpleImputer(strategy="median")
imputer.fit(X)
X = imputer.transform(X)

# Average CV score on the training set was: 0.7122453860565233
exported_pipeline = XGBClassifier(learning_rate=0.1, max_depth=6, min_child_weight=6, n_estimators=100, n_jobs=1, subsample=0.2, verbosity=0)

# exported_pipeline.fit(training_features, training_target)
# results = exported_pipeline.predict(testing_features)

scoring = ['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(exported_pipeline, X, Y, scoring=scoring, cv=20, return_train_score=False)

sorted(scores.keys())

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


['fit_time',
 'score_time',
 'test_accuracy',
 'test_f1_macro',
 'test_precision_macro',
 'test_recall_macro']

In [ ]:
print("mean of test_accuracy : ", np.mean(scores['test_accuracy']))
print("mean of test_precision_macro : ", np.mean(scores['test_precision_macro']))
print("mean of test_recall_macro : ", np.mean(scores['test_recall_macro']))
print("mean of test_f1_macro : ", np.mean(scores['test_f1_macro']))

mean of test_accuracy :  0.7125325158340851
mean of test_precision_macro :  0.7161783187225754
mean of test_recall_macro :  0.7124486393471364
mean of test_f1_macro :  0.7112710755530419


# Dados desbalanceados - Sem frequência


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving todos_dados_prf_com_frequencia_teste.csv to todos_dados_prf_com_frequencia_teste.csv
Saving todos_dados_prf_com_frequencia_treino.csv to todos_dados_prf_com_frequencia_treino.csv
Saving todos_dados_prf_teste.csv to todos_dados_prf_teste.csv
Saving todos_dados_prf_treino.csv to todos_dados_prf_treino.csv


In [ ]:
#Dados de treino
treino = pd.read_csv("todos_dados_prf_treino.csv")
treino = treino.sample(frac=1)

treino.drop(treino.columns[[0,1,9]], axis=1, inplace=True)

#Dados de teste
teste = pd.read_csv("todos_dados_prf_teste.csv")
teste = teste.sample(frac=1)

teste.drop(teste.columns[[0,1,9]], axis=1, inplace=True)

X = pd.concat([treino, teste])


Y = X[X.columns[7]]
X.drop(X.columns[[7]], axis=1, inplace=True)


X.head()

,dia_semana_num,turno_simples,tipo_pista_simples,categoria_sentido_via,tracado_via_simples,condicao_metereologica_simples,tipo_acidente_simples
1231721,4,0,0.0,1,1,1,1
86632,1,1,0.0,1,1,1,1
849008,2,1,0.0,1,2,1,2
129902,4,1,0.0,2,1,1,1
1063631,5,1,1.0,1,1,2,2


## TPOT Default

In [ ]:
imputer = SimpleImputer(strategy="median")
imputer.fit(X)
X = imputer.transform(X)

# Average CV score on the training set was: 0.8376724865759346
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=MultinomialNB(alpha=100.0, fit_prior=True)),
    DecisionTreeClassifier(criterion="entropy", max_depth=7, min_samples_leaf=15, min_samples_split=14)
)

scoring = ['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(exported_pipeline, X, Y, scoring=scoring, cv=20, return_train_score=False)

sorted(scores.keys())

['fit_time',
 'score_time',
 'test_accuracy',
 'test_f1_macro',
 'test_precision_macro',
 'test_recall_macro']

In [ ]:
print("mean of test_accuracy : ", np.mean(scores['test_accuracy']))
print("mean of test_precision_macro : ", np.mean(scores['test_precision_macro']))
print("mean of test_recall_macro : ", np.mean(scores['test_recall_macro']))
print("mean of test_f1_macro : ", np.mean(scores['test_f1_macro']))

mean of test_accuracy :  0.8306636975264559
mean of test_precision_macro :  0.8122681855934812
mean of test_recall_macro :  0.5000289501865176
mean of test_f1_macro :  0.4539936905968016


## TPOT Sparse

In [ ]:
imputer = SimpleImputer(strategy="median")
imputer.fit(X)
X = imputer.transform(X)

# Average CV score on the training set was: 0.8369187126405728
exported_pipeline = BernoulliNB(alpha=1.0, fit_prior=True)

#exported_pipeline.fit(training_features, training_target)
#results = exported_pipeline.predict(testing_features)

scoring = ['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(exported_pipeline, X, Y, scoring=scoring, cv=20, return_train_score=False)

sorted(scores.keys())

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['fit_time',
 'score_time',
 'test_accuracy',
 'test_f1_macro',
 'test_precision_macro',
 'test_recall_macro']

In [ ]:
print("mean of test_accuracy : ", np.mean(scores['test_accuracy']))
print("mean of test_precision_macro : ", np.mean(scores['test_precision_macro']))
print("mean of test_recall_macro : ", np.mean(scores['test_recall_macro']))
print("mean of test_f1_macro : ", np.mean(scores['test_f1_macro']))

mean of test_accuracy :  0.8301060648607519
mean of test_precision_macro :  0.7195021151693467
mean of test_recall_macro :  0.49961960269799305
mean of test_f1_macro :  0.4536434185916513


# Dados desbalanceados - Com frequência


In [ ]:
#Dados de treino
treino = pd.read_csv("todos_dados_prf_com_frequencia_treino.csv")
treino = treino.sample(frac=1)

treino.drop(treino.columns[[0,1,9]], axis=1, inplace=True)

#Dados de teste
teste = pd.read_csv("todos_dados_prf_com_frequencia_teste.csv")
teste = teste.sample(frac=1)

teste.drop(teste.columns[[0,1,9]], axis=1, inplace=True)

X = pd.concat([treino, teste])


Y = X[X.columns[7]]
X.drop(X.columns[[7]], axis=1, inplace=True)


X.head()

,dia_semana_num,turno_simples,tipo_pista_simples,categoria_sentido_via,tracado_via_simples,condicao_metereologica_simples,tipo_acidente_simples,frequencia
165186,3,0,0.0,1,2,1,1,0.000011
284368,3,1,0.0,1,2,2,2,0.000059
1375808,6,0,1.0,2,1,1,1,0.000325
427751,4,0,1.0,2,1,1,1,0.000084
426234,7,1,1.0,1,2,0,1,0.000539


## TPOT Default

In [ ]:
imputer = SimpleImputer(strategy="median")
imputer.fit(X)
X = imputer.transform(X)

# Average CV score on the training set was: 0.8307791023606093
exported_pipeline = DecisionTreeClassifier(criterion="gini", max_depth=4, min_samples_leaf=16, min_samples_split=4)

# exported_pipeline.fit(training_features, training_target)
# results = exported_pipeline.predict(testing_features)

scoring = ['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(exported_pipeline, X, Y, scoring=scoring, cv=20, return_train_score=False)

sorted(scores.keys())

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Pr

['fit_time',
 'score_time',
 'test_accuracy',
 'test_f1_macro',
 'test_precision_macro',
 'test_recall_macro']

In [ ]:
print("mean of test_accuracy : ", np.mean(scores['test_accuracy']))
print("mean of test_precision_macro : ", np.mean(scores['test_precision_macro']))
print("mean of test_recall_macro : ", np.mean(scores['test_recall_macro']))
print("mean of test_f1_macro : ", np.mean(scores['test_f1_macro']))

mean of test_accuracy :  0.8307782502780403
mean of test_precision_macro :  0.41538912513902015
mean of test_recall_macro :  0.5
mean of test_f1_macro :  0.45378420360197225


## TPOT Sparse

In [ ]:
from xgboost import XGBClassifier

In [ ]:
imputer = SimpleImputer(strategy="median")
imputer.fit(X)
X = imputer.transform(X)

# Average CV score on the training set was: 0.830865556718787
exported_pipeline = XGBClassifier(learning_rate=0.5, max_depth=1, min_child_weight=8, n_estimators=100, n_jobs=1, subsample=0.5, verbosity=0)

# exported_pipeline.fit(training_features, training_target)
# results = exported_pipeline.predict(testing_features)

scoring = ['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(exported_pipeline, X, Y, scoring=scoring, cv=20, return_train_score=False)

sorted(scores.keys())

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


['fit_time',
 'score_time',
 'test_accuracy',
 'test_f1_macro',
 'test_precision_macro',
 'test_recall_macro']

In [ ]:
print("mean of test_accuracy : ", np.mean(scores['test_accuracy']))
print("mean of test_precision_macro : ", np.mean(scores['test_precision_macro']))
print("mean of test_recall_macro : ", np.mean(scores['test_recall_macro']))
print("mean of test_f1_macro : ", np.mean(scores['test_f1_macro']))

mean of test_accuracy :  0.8308609222602398
mean of test_precision_macro :  0.6972506251256265
mean of test_recall_macro :  0.5010655512727342
mean of test_f1_macro :  0.4563258363217252
